# Global preparations

In [1]:
import sys
import os
import matplotlib.pyplot as plt
import time
import torch
import torchvision
import logging
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision.transforms.functional import to_pil_image
from PIL import Image
from tqdm.notebook import tqdm

sys.path.insert(0, os.path.expanduser('~/spiky'))
device = 'cuda:7'
summation_dtype = torch.float32
random_seed = 1
torch.manual_seed(random_seed)
np.random.seed(random_seed)
torch.backends.cudnn.enabled = True
logger = logging.getLogger('stdout_logger')
logger.setLevel(logging.INFO)
if not logger.hasHandlers():
    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setFormatter(logging.Formatter(fmt='%(asctime)s|%(levelname)s|%(message)s'))
    logger.addHandler(stdout_handler)

# MNIST preparation

Let's read the MNIST dataset and represent it in sparse form

In [2]:
mnist_dataset_dir = 'mnist'
mnist_train_dataset = torchvision.datasets.MNIST(
    mnist_dataset_dir, train=True, download=True
)
mnist_test_dataset = torchvision.datasets.MNIST(
    mnist_dataset_dir, train=False, download=True
)
mnist_train_data_device = mnist_train_dataset.data.to(device=device).to(dtype=torch.float32) / 255
mnist_test_data_device = mnist_test_dataset.data.to(device=device).to(dtype=torch.float32) / 255
mnist_train_targets_device = mnist_train_dataset.targets.to(device=device)
mnist_test_targets_device = mnist_test_dataset.targets.to(device=device)

mnist_train_dataset = TensorDataset(
    mnist_train_data_device, mnist_train_targets_device
)
mnist_test_dataset = TensorDataset(
    mnist_test_data_device, mnist_test_targets_device
)

In [3]:
mnist_test_loader = torch.utils.data.DataLoader(
    mnist_train_dataset,
    batch_size=1, shuffle=True
)

example_data, example_targets = next(iter(mnist_test_loader))
to_pil_image(example_data[0])

In [4]:
patch_h = patch_w = 28
input_shape = (patch_h, patch_w)
clustering_dim = 44

In [5]:
example_data.shape

torch.Size([1, 28, 28])

In [6]:
# HYPERPLANE preprocessing
# # x = example_data
# # x = x.reshape(x.shape[0], 28 * 28)
# # m = (torch.rand(28 * 28, 2048 * 10, device=device) - 0.5) * 2
# # (x @ m).shape
# # m[:,torch.arange(0, 1024 * 10) * 2 + 1] = 0
# mnist_train_data_device = mnist_train_data_device.reshape(mnist_train_data_device.shape[0], 28 * 28) @ m
# mnist_test_data_device = mnist_test_data_device.reshape(mnist_test_data_device.shape[0], 28 * 28) @ m
# mnist_train_dataset = TensorDataset(
#     mnist_train_data_device, mnist_train_targets_device
# )
# mnist_test_dataset = TensorDataset(
#     mnist_test_data_device, mnist_test_targets_device
# )

# Baseline MLP

In [7]:
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

class BaselineMLP(nn.Module):
    def __init__(self, device, n_inputs, hidden_dim):
        super().__init__()
        self.fc1 = nn.Linear(n_inputs, hidden_dim, bias=False, device=device)
        self.fc2 = nn.Linear(hidden_dim, 10, bias=False, device=device)
        
    def forward(self, x):
        x1 = x / (x.norm(dim=-1, keepdim=True) + 1e-16)
        x2 = F.relu(self.fc1(x1))
        return self.fc2(x2)

In [8]:
baseline_net = BaselineMLP(device, input_shape[0] * input_shape[1], clustering_dim**2)

In [ ]:
optimizer = optim.Adam(baseline_net.parameters(), lr=0.001)

loss_func = nn.CrossEntropyLoss()
for epoch in range(16):
    baseline_net.train()
    train_loader = torch.utils.data.DataLoader(mnist_train_dataset, batch_size=64, shuffle=True)
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        target_one_hot = torch.nn.functional.one_hot(target, num_classes=10)
        optimizer.zero_grad()
        output = baseline_net(data.reshape([data.shape[0], input_shape[0] * input_shape[1]]))
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    train_acc = 100. * correct / len(train_loader.dataset)
    
    baseline_net.eval()
    correct = 0
    with torch.no_grad():
        test_loader = torch.utils.data.DataLoader(mnist_test_dataset, batch_size=256, shuffle=True)
        for data, target in test_loader:
            output = baseline_net(data.reshape([data.shape[0], input_shape[0] * input_shape[1]]))
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_acc = 100. * correct / len(test_loader.dataset)
    print(f'Epoch {epoch+1} Train/test accuracy: {train_acc:.2f}/{test_acc:.2f}')

# Simple LUT network

In [9]:
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

from spiky.lut.LUTLayer import (
    Conv2DLUTLayer, LUTLayer, LUTSharedContext, SynapseMeta, GradientPolicy, GradientType
)
from spiky.util.torch_utils import make_lr_getter

synapse_meta = SynapseMeta(
    min_weight=-1.0,
    max_weight=1.0,
    initial_weight=0.0,
    initial_noise_level=0.0
)

shared_lut_ctx = LUTSharedContext()
shared_lut_ctx.to_device(device)
g_policy = GradientPolicy(GradientType.Internal, normalized=False)

lut_net = None
baseline_net = None
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [10]:
class LutNet(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.filter_lut = LUTLayer(
            n_inputs=28 * 28,
            n_anchors_per_detector=10,
            n_detectors=1024,
            n_outputs=128,
            synapse_meta=synapse_meta,
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            summation_dtype=summation_dtype,
            # _explicit_anchors=torch.arange(0, 20480, dtype=torch.int32), # for hyperplanes
            random_seed=random_seed,
            device=device
        )
        print('filter finished')
        l = []
        for i in range(10):
            l.append(
                LUTLayer(
                    n_inputs=128,
                    n_anchors_per_detector=16,
                    n_detectors=128,
                    n_outputs=128,
                    synapse_meta=synapse_meta,
                    weights_gradient_policy=g_policy,
                    shared_context=shared_lut_ctx,
                    summation_dtype=summation_dtype,
                    random_seed=random_seed,
                    device=device
                )
            )
            print(f'conv_lut {i} finished')
        self.conv_luts = nn.ModuleList(l)
        self.final_lut = LUTLayer(
            n_inputs=128,
            n_anchors_per_detector=16,
            n_detectors=128,
            n_outputs=10,
            synapse_meta=synapse_meta,
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            device=device
        ) 

    def forward(self, x):
        B = x.shape[0]
        x = self.filter_lut(x.reshape(B, 28 * 28))
        # x = self.filter_lut(x)
        for c_lut in self.conv_luts:
            x = c_lut(x) + x
        return self.final_lut(x)
    
    def setup_external_learning_rate_hook(self, optimizer):
        lr_getter = make_lr_getter(optimizer)
        self.filter_lut.set_external_learning_rate_hook(lr_getter)
        for c_lut in self.conv_luts:
            c_lut.set_external_learning_rate_hook(lr_getter)
        self.final_lut.set_external_learning_rate_hook(lr_getter)

In [11]:
lut_net = LutNet(device)
lut_net

filter finished
conv_lut 0 finished
conv_lut 1 finished
conv_lut 2 finished
conv_lut 3 finished
conv_lut 4 finished
conv_lut 5 finished
conv_lut 6 finished
conv_lut 7 finished
conv_lut 8 finished
conv_lut 9 finished


LutNet(
  (filter_lut): LUTLayer(784 inputs, 1024 detectors, 128 outputs, 10 anchors per detector)
  (conv_luts): ModuleList(
    (0-9): 10 x LUTLayer(128 inputs, 128 detectors, 128 outputs, 16 anchors per detector)
  )
  (final_lut): LUTLayer(128 inputs, 128 detectors, 10 outputs, 16 anchors per detector)
)

In [12]:
from torch.optim.lr_scheduler import LambdaLR

def lr_func(t):
    return min(
        1.0 / (1 + t)**0.5,
        (t / 4000.0) / 4000.0**0.5
    )

optimizer = optim.SGD(lut_net.parameters(), lr=1.0)
sched = LambdaLR(optimizer, lr_lambda=lr_func)
lut_net.setup_external_learning_rate_hook(optimizer)
# optimizer = optim.Adam(lut_net.parameters(), lr=0.001)

In [13]:
n_epochs = 8
batch_size = 256
with torch.cuda.device(device): 
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(n_epochs):
        lut_net.train()
        train_loader = torch.utils.data.DataLoader(mnist_train_dataset, batch_size=batch_size, shuffle=True)
        correct = 0
        pbar = tqdm(total=len(train_loader))
        for batch_idx, (data, target) in enumerate(train_loader):
            target_one_hot = torch.nn.functional.one_hot(target, num_classes=10)
            optimizer.zero_grad()
            output = lut_net(data)
            loss = loss_func(output, target)
            loss.backward()
            optimizer.step()
            sched.step()
            pbar.update(1)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            if (batch_idx % 100) == 0:
                pbar.set_description(
                    f"Epoch {epoch + 1}/{n_epochs}"
                    f", lr {sched.get_last_lr()[0]:.4f}"
                )
            
        train_acc = 100. * correct / len(train_loader.dataset)

        lut_net.eval()
        correct = 0
        with torch.no_grad():
            test_loader = torch.utils.data.DataLoader(mnist_test_dataset, batch_size=256, shuffle=True)
            for data, target in test_loader:
                output = lut_net(data)
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
        test_acc = 100. * correct / len(test_loader.dataset)
        print(f'Epoch {epoch+1} Train/test accuracy: {train_acc:.2f}/{test_acc:.2f}, lr {sched.get_last_lr()[0]:.4f}')

  0%|          | 0/235 [00:00<?, ?it/s]

Epoch 1 Train/test accuracy: 49.25/56.18, lr 0.0009


  0%|          | 0/235 [00:00<?, ?it/s]

Epoch 2 Train/test accuracy: 67.20/80.75, lr 0.0019


  0%|          | 0/235 [00:00<?, ?it/s]

...

Epoch 6 Train/test accuracy: 96.81/95.84, lr 0.0056


  0%|          | 0/235 [00:00<?, ?it/s]

Epoch 7 Train/test accuracy: 97.59/96.53, lr 0.0065


  0%|          | 0/235 [00:00<?, ?it/s]

Epoch 8 Train/test accuracy: 98.16/96.64, lr 0.0074


In [14]:
print(lut_net.filter_lut.get_profiling_stats())

connections_manager::add_connections: 10.6387 ms / 1 = 10.6387 ms
connections_manager::add_connections::estimate_capacity: 4.47858 ms / 1 = 4.47858 ms
connections_manager::add_connections::create_groups: 1.66806 ms / 1 = 1.66806 ms
connections_manager::finalize_groups: 10.9148 ms / 1 = 10.9148 ms
connections_manager::finalize_groups::gather_forward_info: 0.06224 ms / 1 = 0.06224 ms
connections_manager::finalize_groups::shuffle_groups: 0 ms / 0 = -nan ms
connections_manager::finalize_groups::calculate_backward_stats: 3.04101 ms / 1 = 3.04101 ms
connections_manager::finalize_groups::reduce_backward_stats: 0.04936 ms / 1 = 0.04936 ms
connections_manager::finalize_groups::calculate_backward_counters: 3.76902 ms / 1 = 3.76902 ms
connections_manager::finalize_groups::reduce_backward_counters: 0.05058 ms / 1 = 0.05058 ms
connections_manager::finalize_groups::reduce_backward_capacity: 0.05923 ms / 1 = 0.05923 ms
connections_manager::finalize_groups::distribute_big: 0.044 ms / 1 = 0.044 ms
conn

In [ ]:
from spiky.util.visual_helpers import grayscale_to_red_and_blue

output_shape = andn_net.layer1.output_shape()
target_image = torch.zeros([output_shape[0] * (patch_h + 2), output_shape[1] * (patch_w + 2)])
weights = andn_net.layer1.export_weights()

for i in range(output_shape[0]):
    for j in range(output_shape[1]):
        target_image[
            i * (patch_h + 2) + 1:i * (patch_h + 2) + patch_h + 1,
            j * (patch_w + 2) + 1:j * (patch_w + 2) + patch_w + 1
        ] = weights[i, j]

img = grayscale_to_red_and_blue(target_image.unsqueeze(0))
color_rescaler = 1.0 / (img.abs().max() + 0.0000000001)
img = to_pil_image((img * color_rescaler).clip(0.0, 1.0))
img.resize(
    [
        output_shape[0] * receptive_field_shape[0], 
        output_shape[1] * receptive_field_shape[1]
    ], Image.NEAREST
)

# Layered ANDN network

In [ ]:
from spiky.andn.ANDNLayer import Conv2DANDNLayer, SynapseMeta

input_shape=(28, 28)
final_output_shape = (1, 10)
cl_d = 6
rf_d = 5
synapse_meta_1 = SynapseMeta(
    min_weight=0.0,
    max_weight=10.0,
    initial_weight=1.0,
    initial_noise_level=-10.0
)

synapse_meta_2 = SynapseMeta(
    min_weight=-1.0,
    max_weight=1.0,
    initial_weight=1.0,
    initial_noise_level=-2.0
)

class ANDNNet(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.layer1 = Conv2DANDNLayer(
            input_shape=input_shape,
            inhibition_grid_shape=None,
            receptive_field_shape=(rf_d, rf_d),
            receptive_field_stride_shape=(1, 1),
            output_kernel_shape=(cl_d, cl_d),
            backprop_hebb_ratio_on_torch_backward=1.0,
            synapse_meta=synapse_meta_1,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            device=device
        )
        self.layer2 = Conv2DANDNLayer(
            input_shape=self.layer1.output_shape(),
            inhibition_grid_shape=(cl_d, cl_d),
            receptive_field_shape=(rf_d * cl_d, rf_d * cl_d),
            receptive_field_stride_shape=(cl_d, cl_d),
            output_kernel_shape=(cl_d, cl_d),
            backprop_hebb_ratio_on_torch_backward=1.0,
            synapse_meta=synapse_meta_1,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            device=device
        )
        self.layer1.set_descendant_andn_layer(self.layer2)
        self.layer3 = Conv2DANDNLayer(
            input_shape=self.layer2.output_shape(),
            inhibition_grid_shape=(cl_d, cl_d),
            receptive_field_shape=self.layer2.output_shape(),
            receptive_field_stride_shape=self.layer2.output_shape(),
            output_kernel_shape=final_output_shape,
            backprop_hebb_ratio_on_torch_backward=0.0,
            synapse_meta=synapse_meta_2,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            device=device
        )
        self.layer2.set_descendant_andn_layer(self.layer3)

    def forward(self, x):
        x = x / (x.norm(dim=(-1, -2), keepdim=True) + 1e-16)
        return self.layer3(self.layer2(self.layer1(x))).squeeze(1)

In [ ]:
layered_andn_net = ANDNNet(device)
layered_andn_net

In [ ]:
optimizer = optim.Adam(layered_andn_net.parameters(), lr=0.001)

loss_func = nn.CrossEntropyLoss()
for epoch in range(16):
    layered_andn_net.train()
    train_loader = torch.utils.data.DataLoader(mnist_train_dataset, batch_size=64, shuffle=True)
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        target_one_hot = torch.nn.functional.one_hot(target, num_classes=10)
        optimizer.zero_grad()
        output = layered_andn_net(data)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    train_acc = 100. * correct / len(train_loader.dataset)
    
    layered_andn_net.eval()
    correct = 0
    with torch.no_grad():
        test_loader = torch.utils.data.DataLoader(mnist_test_dataset, batch_size=256, shuffle=True)
        for data, target in test_loader:
            output = layered_andn_net(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_acc = 100. * correct / len(test_loader.dataset)
    print(f'Epoch {epoch+1} Train/test accuracy: {train_acc:.2f}/{test_acc:.2f}')

In [ ]:
from spiky.util.visual_helpers import grayscale_to_red_and_blue

output_shape = layered_andn_net.layer1.output_shape()
patch_h, patch_w = (rf_d, rf_d)
target_image = torch.zeros([output_shape[0] * (rf_d + 2), output_shape[1] * (rf_d + 2)])
weights = layered_andn_net.layer1.export_weights()

for i in range(output_shape[0]):
    for j in range(output_shape[1]):
        target_image[
            i * (rf_d + 2) + 1:i * (rf_d+ 2) + rf_d + 1,
            j * (rf_d + 2) + 1:j * (rf_d + 2) + rf_d + 1
        ] = weights[i, j]

img = grayscale_to_red_and_blue(target_image.unsqueeze(0))
color_rescaler = 1.0 / (img.abs().max() + 0.0000000001)
img = to_pil_image((img * color_rescaler).clip(0.0, 1.0))
img.resize(
    [
        output_shape[0] * rf_d * 4, 
        output_shape[1] * rf_d * 4
    ], Image.NEAREST
)

# JUNK

## Explicit connections LUT (for performance test)

In [ ]:
class LutNet(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.filter_lut = Conv2DLUTLayer(
            input_shape=(28, 28),
            n_anchors_per_detector=10,
            detectors_shape=(32, 32),
            output_kernel_shape=(12, 12),
            receptive_field_shape=(28, 28),
            receptive_field_stride_shape=(28, 28),
            lut_receptive_field_shape=(32, 32),
            lut_receptive_field_stride_shape=(32, 32),
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            synapse_meta=synapse_meta,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            _max_groups_in_growth_buffer=10**7,
            device=device
        )
        print('filter finished')
        l = []
        for i in range(10):
            l.append(
                Conv2DLUTLayer(
                    input_shape=(12, 12),
                    n_anchors_per_detector=14,
                    detectors_shape=(12, 12),
                    output_kernel_shape=(12, 12),
                    sequence_length=1,
                    receptive_field_shape=(12, 12),
                    receptive_field_stride_shape=(12, 12),
                    lut_receptive_field_shape=(12, 12),
                    lut_receptive_field_stride_shape=(12, 12),
                    weights_gradient_policy=g_policy,
                    shared_context=shared_lut_ctx,
                    synapse_meta=synapse_meta,
                    summation_dtype=summation_dtype,
                    random_seed=random_seed,
                    device=device
                )
            )
            print(f'conv_lut {i} finished')
        self.conv_luts = nn.ModuleList(l)
        self.final_lut = Conv2DLUTLayer(
            input_shape=(12, 12),
            n_anchors_per_detector=14,
            detectors_shape=(12, 12),
            output_kernel_shape=(1, 10),
            sequence_length=1,
            receptive_field_shape=(12, 12),
            receptive_field_stride_shape=(12, 12),
            lut_receptive_field_shape=(12, 12),
            lut_receptive_field_stride_shape=(12, 12),
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            synapse_meta=synapse_meta,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            device=device
        ) 

    def forward(self, x):
        B = x.shape[0]
        x = self.filter_lut(x)
        for c_lut in self.conv_luts:
            x = c_lut(x) + x
        return self.final_lut(x).reshape(B, 10)
    
    def setup_external_learning_rate_hook(self, optimizer):
        lr_getter = make_lr_getter(optimizer)
        self.filter_lut.set_external_learning_rate_hook(lr_getter)
        for c_lut in self.conv_luts:
            c_lut.set_external_learning_rate_hook(lr_getter)
        self.final_lut.set_external_learning_rate_hook(lr_getter)

# Profiling

In [ ]:
with torch.cuda.device(device):
    with torch.autograd.profiler.profile(use_cuda=True) as prof:
        with torch.profiler.record_function("Dense to sparse conversion"):
            torch.cuda.synchronize(device)
            t0 = time.time()
            indices, values = ds_conv_new.dense_to_sparse_32(
                t, True, 
                shared_context.get_densify_buffers(5000000, torch.device(device))
            )
            torch.cuda.synchronize(device)
            print((time.time() - t0) * 1000, "ms")
            t0 = time.time()
            st = torch.sparse_coo_tensor(
                indices=indices.unsqueeze(0),
                values=values,
                size=indices.shape,
                is_coalesced=True
            )
            print((time.time() - t0) * 1000, "ms")
            torch.cuda.synchronize(device)
            print(ds_conv_new.get_profiling_stats())
print(prof.key_averages().table(sort_by="cuda_time_total"))

In [ ]:
from spiky.util.torch_utils import DenseToSparseConverter
from spiky.lut.LUTLayer import LUTSharedContext


with torch.cuda.device(device):
    torch.cuda.synchronize(device)
    t0 = time.time()
    indices, values = ds_conv_new.dense_to_sparse_32(
        t, True, 
        shared_context.get_densify_buffers(5000000, torch.device(device))
    )
    torch.cuda.synchronize(device)
    print((time.time() - t0) * 1000, "ms")
    t0 = time.time()
    st = torch.sparse_coo_tensor(
        indices=indices.unsqueeze(0),
        values=values,
        size=indices.shape,
        is_coalesced=True
    )
    print((time.time() - t0) * 1000, "ms")
    torch.cuda.synchronize(device)
    print(ds_conv_new.get_profiling_stats())

In [ ]:
fwd_total = 0.0
bwd_total = 0.0
opt_total = 0.0
count = 0

with torch.cuda.device(device): 
    start_fwd = torch.cuda.Event(enable_timing=True)
    end_fwd   = torch.cuda.Event(enable_timing=True)
    start_bwd = torch.cuda.Event(enable_timing=True)
    end_bwd   = torch.cuda.Event(enable_timing=True)
    start_opt = torch.cuda.Event(enable_timing=True)
    end_opt   = torch.cuda.Event(enable_timing=True)

In [ ]:
# with torch.autograd.profiler.profile(use_cuda=True) as prof:
with torch.cuda.device(device): 
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(1):
        lut_net.train()
        train_loader = torch.utils.data.DataLoader(mnist_train_dataset, batch_size=64, shuffle=True)
        correct = 0
        for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
            target_one_hot = torch.nn.functional.one_hot(target, num_classes=10)
            count += 1

            optimizer.zero_grad()

            # forward
            start_fwd.record()
            output = lut_net(data)
            end_fwd.record()
            torch.cuda.synchronize()
            fwd_ms = start_fwd.elapsed_time(end_fwd)
            fwd_total += fwd_ms

            # backward
            loss = loss_func(output, target)
            start_bwd.record()
            loss.backward()
            end_bwd.record()
            torch.cuda.synchronize()
            bwd_ms = start_bwd.elapsed_time(end_bwd)
            bwd_total += bwd_ms

            # optimizer
            start_opt.record()
            optimizer.step()
            end_opt.record()
            torch.cuda.synchronize()
            opt_ms = start_opt.elapsed_time(end_opt)
            opt_total += opt_ms

            sched.step()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            if batch_idx == 128:
                break
        train_acc = 100. * correct / len(train_loader.dataset)
        break
        lut_net.eval()
        correct = 0
        with torch.no_grad():
            test_loader = torch.utils.data.DataLoader(mnist_test_dataset, batch_size=256, shuffle=True)
            for data, target in test_loader:
                output = lut_net(data)
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
        test_acc = 100. * correct / len(test_loader.dataset)
        print(f'Epoch {epoch+1} Train/test accuracy: {train_acc:.2f}/{test_acc:.2f}, lr {sched.get_last_lr()[0]:.4f}')

In [ ]:
avg_fwd = fwd_total / count
avg_bwd = bwd_total / count
avg_opt = opt_total / count
print(f"Avg fwd/bwd/opt: {avg_fwd:.3f} / {avg_bwd:.3f} / {avg_opt:.3f} ms")

In [ ]:
print(prof.key_averages().table(sort_by="cuda_time_total"))

In [ ]:
print(lut_net.filter_lut.get_profiling_stats())

In [ ]:
print(lut_net.conv_luts[6].get_profiling_stats())

In [ ]:
print(lut_net.final_lut.get_profiling_stats())